In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


### Scanner Tests for Exception

In [ ]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

In [ ]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

### Compiler Tests for Exception

### Explicit Exception:

In [ ]:
# "Throw" can be compiled，but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""", 'test.wat')

In [ ]:
!wat2wasm test.wat

In [ ]:
#"try-catch" can be applied, but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch done := true
""", 'test2.wat')

In [ ]:
!wat2wasm test2.wat

### Implicit Exception

In [2]:
# Implicit Exception: Index out of bound
# == if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""
var a: [1..2] → integer
program fool
    var i: integer
        a[1] := 1; a[2] := 2
        i := 2
        write(a[i])
""", 'badindex.wat')

Importing Jupyter notebook from CGwat.ipynb


In [3]:
!wat2wasm badindex.wat

badindex.wat:27:1: error: opcode not allowed: throw
throw 0
^^^^^


In [ ]:
runpywasm("badindex.wasm")

In [ ]:
# Implicit Exception: Division by 0
compileString("""
program fool
    var a, x: integer
        x := 0
        a := 3 div x
        write(a)
""")

In [ ]:
!wat2wasm divby0.wat

In [ ]:
runpywasm("divby0.wasm")